# Problem Set 4

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import scipy.stats as stats
import math
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Data Import

In [2]:
#first read the stata file (for data)
data = pd.read_stata('G:\My Drive\Computational Economics\PS4_data.dta')

#data
data.head(n=10)
#summary statistics
data.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


# Question1:Select only male heads of household who are between 25 and 60 years of age and earn wages > $7/hr.

In [3]:
#  data Cleaning
# Limiting the sample to only male heads
data = data[data.hsex == 1 ]

# Define the min and max ages

minage = 25
maxage = 60

years = [1971, 1980, 1990, 2000]

# Limit to only those making > $7/hr
minwage = 7

# Question2:Create indicator and continuous variables as necessary

In [4]:
# Create log wage variable of male head
data['hwage'] = data['hlabinc'] / data['hannhrs']
data['lhwage'] = np.log(data['hwage'])

# Create dummy variables for race
data['black'] = (data['hrace'] == 2).astype(int)
data['hispanic'] = (data['hrace'] == 5).astype(int)
data['otherrace'] = ((data['hrace'] != 1) & (data['hrace'] != 2) & (data['hrace'] != 5)).astype(int)

# create age squared and constant
data['agesqr'] = (data['age']*data['age'])

# drop missing values
data = data.loc[data['lhwage'].isna() != True]
data = data.loc[data['hlabinc'].isna() != True]
data = data.loc[data['hannhrs'].isna() != True]
data = data.loc[data['hsex'].isna() != True]
data = data.loc[data['hrace'].isna() != True]
data = data.loc[data['hyrsed'].isna() != True]


#Keep only the relevent variables for analysis
data = data[ ['lhwage', 'hwage', 'hlabinc', 'hannhrs', 'hyrsed', 'age', 'agesqr', 'black', 'hispanic', 'otherrace', 'year' ] ]

# Limit the data to just the relevant values
data = data[ (data.hwage > minwage) & (data.age >= minage) & \
                    (data.age <= maxage) & (data.lhwage != np.inf)]

#constant
data['const'] = 1

 # Rechecks the data and variables
data.describe()  


,lhwage,hwage,hlabinc,hannhrs,hyrsed,age,agesqr,black,hispanic,otherrace,year,const
count,57062.000000,57062.000000,5.706200e+04,57062.000000,57062.000000,57062.000000,57062.000000,57062.000000,57062.0,57062.000000,57062.000000,57062.0
mean,3.010804,24.320921,5.282805e+04,2228.557617,13.529967,39.243629,1631.315552,0.056377,0.0,0.022519,1986.575672,1.0
std,0.544096,25.209909,5.236477e+04,620.054077,2.450013,9.578858,791.953796,0.230651,0.0,0.148367,8.712311,0.0
min,1.945946,7.000252,1.666980e+01,2.000000,1.000000,25.000000,625.000000,0.000000,0.0,0.000000,1971.000000,1.0
25%,2.635527,13.950662,3.037345e+04,1952.000000,12.000000,31.000000,961.000000,0.000000,0.0,0.000000,1979.000000,1.0
50%,2.991457,19.914677,4.382858e+04,2160.000000,13.000000,38.000000,1444.000000,0.000000,0.0,0.000000,1987.000000,1.0
75%,3.324710,27.790929,6.138495e+04,2519.000000,16.000000,47.000000,2209.000000,0.000000,0.0,0.000000,1994.000000,1.0
max,7.448526,1717.330322,3.771521e+06,5840.000000,17.000000,60.000000,3600.000000,1.000000,0.0,1.000000,2002.000000,1.0


# Question3:Estimate the following model via a Maximum Likelihood Estimator separately for t = 1971, 1980, 1990,2000:


Model:
$\ln \left(w_{i, t}\right)=\alpha+\beta_{1} E d u c_{i, t}+\beta_{2}$ Age $_{i, t}+\beta_{3} A g e_{i, t}^{2}+\beta_{4}$ Black $_{i, t}+\beta_{5}$ Hispanic $_{i, t}+\beta_{6}$ OtherRace $_{i, t}+\varepsilon_{i, t}$  

where,

$w_{i, t}=$ wage of individual $i$ in survey year $t$

$E d u c_{i, t}=$ education in years

$A g e_{i, t}=$ age in years

Black$_{i, t}$, Hispanic$_{i, t}$, OtherRace$_{i, t}=$ dummy variables for race $=$ Black, Hispanic, Not $\in\{$ White, Black, Hispanic $\}$.

# Maximum Likelihood Estimator

In [5]:
# Define function to compute the log likelihood
def model_ML(b, data, t):
    '''
    Compute the log likelihood
    
    Args:
        B: a vector of coefficient values
        sigma (float): the standard deviation of the error term of the model
        data: a dataframe of the data to be used
        t: the year(s) of interest
        
    Returns:
        negLL (float): the negative Log Likelihood
    '''
    mdata = data[data['year'].isin(t)]
    # Did not include a coefficient for 'Hispanic' b/c there were no observations, so it was not used in the model
    const = b[0]
    B1 = b[1]
    B2 = b[2]
    B3 = b[3]
    B4 = b[4]
    B5 = b[5]
    sigma = b[6]
    
    N = len(mdata['lhwage'])
    
    # The linear model
    lhwage = const + B1 * mdata['hyrsed'] + B2 * mdata['age']+ B3 * mdata['agesqr'] + B4 * mdata['black'] + B5 * mdata['otherrace']
    
    #Note: We didnot include the variable 'hispanic' beacuse it does not have variation in our sample data.
    
    
    # The error term for getting Log Likelihood
    e = mdata['lhwage'] - lhwage
    
    # Calculate negative Log Likelihood
    negLL = (N/2) * np.log(sigma ** 2) + (N/2) * np.log(2 * math.pi) + (np.sum(e ** 2)) / (2 * sigma ** 2)
    
    return(negLL)

In [6]:

guess = [1, 0.06, 0.04, -0.02, -0.03, 0.01, 0.5] # initial coefficient guesses
Beta = []
for t in years:
    params = (data, [t])
    results = opt.minimize(model_ML, guess, args = params , method = 'Nelder-Mead', tol = 1e-10, options = {'maxiter': 5000})
    Beta.append(results.x)
    print(t, results.x)


1971 [ 1.40449172e+00  6.23410400e-02  2.64122605e-02 -1.65202167e-04
 -6.48042025e-02  7.97693518e-03  4.10701053e-01]
1980 [ 7.67208189e-01  6.53130226e-02  5.79213467e-02 -5.47367069e-04
 -4.78709230e-02  1.40351216e-02  4.48177361e-01]
1990 [ 1.12969989e+00  8.58531632e-02  2.14756456e-02 -1.05456900e-04
 -3.96019438e-02  1.47200395e-02  4.84073537e-01]
2000 [ 0.17836886  0.11105372  0.05907242 -0.00058182 -0.01540858  0.02413966
  0.53733816]


# Question4:Interpret the coefficient β1. How do the returns to education change over time in these data?

 We can interpret the $\beta_1$ coefficient as returns to schooling i.e percent change in wage resulting from one additiona year of schooling. For example, for the year 1971, one additional year of schooling increases the person's wage by 6.65 percentage.From the coefficients on education for different years, we can see that the returns to schooling is increasing over time. Between  1971 and 1980, the coefficients on education was almost same, but later it increased to 0.086 in 1990, and to 0.111 in 2020.Thus, between 1980 and 1990 the return to schooling increses by around 2 percentage point, and between 1990 and 2000 it increases by 2.5 percentage point. Overall, we can see that there is a positive trend on returns to schooling.

# Now, lets check the MLE results against an OLS estimator to confirm our results

# OLS regression results

In [7]:
# create datasets for each year of interest
data1971 = data.loc[data['year'] == 1971]
data1980 = data.loc[data['year'] == 1980]
data1990 = data.loc[data['year'] == 1990]
data2000 = data.loc[data['year'] == 2000]

In [8]:
#OLS regression for year=1971
Y1971 = data1971['lhwage']
X1971 = data1971[['const', 'hyrsed', 'age', 'agesqr', 'black', 'otherrace']]

reg1971 = sm.OLS(Y1971,X1971).fit()
print(reg1971.summary())

                            OLS Regression Results                            
Dep. Variable:                 lhwage   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     96.14
Date:                Mon, 04 Oct 2021   Prob (F-statistic):           5.66e-87
Time:                        21:34:35   Log-Likelihood:                -713.71
No. Observations:                1380   AIC:                             1439.
Df Residuals:                    1374   BIC:                             1471.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5863      0.193      3.033      0.0

 # MLE vs OLS result for 1971: In MLE the coefficient on education was 0.0623 but in OLS it is 0.0665; so the resluts are very similar!

In [9]:
#OLS regression for year=1980
Y1980 = data1980['lhwage']
X1980 = data1980[['const', 'hyrsed', 'age', 'agesqr', 'black', 'otherrace']]
reg1980 = sm.OLS(Y1980,X1980).fit()
print(reg1980.summary())

                            OLS Regression Results                            
Dep. Variable:                 lhwage   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     78.10
Date:                Mon, 04 Oct 2021   Prob (F-statistic):           1.80e-74
Time:                        21:34:35   Log-Likelihood:                -1142.4
No. Observations:                1856   AIC:                             2297.
Df Residuals:                    1850   BIC:                             2330.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0023      0.192      5.226      0.0

# MLE vs OLS result for 1980: In MLE the coefficient on education was 0.0653 but in OLS it is 0.0660; so the resluts are very similar!

In [10]:
#OLS regression for year=1990
Y1990 = data1990['lhwage']
X1990 = data1990[['const', 'hyrsed', 'age', 'agesqr', 'black', 'otherrace']]

reg1990 = sm.OLS(Y1990,X1990).fit()
print(reg1990.summary())

                            OLS Regression Results                            
Dep. Variable:                 lhwage   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     115.9
Date:                Mon, 04 Oct 2021   Prob (F-statistic):          7.46e-108
Time:                        21:34:35   Log-Likelihood:                -1385.1
No. Observations:                2013   AIC:                             2782.
Df Residuals:                    2007   BIC:                             2816.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2772      0.217      1.277      0.2

# MLE vs OLS result for 1990: In MLE the coefficient on education was 0.086 but in OLS it is 0.095; so the resluts are very close!

In [11]:
#OLS regression for year=2000
Y2000 = data2000['lhwage']
X2000 = data2000[['const', 'hyrsed', 'age', 'agesqr', 'black', 'otherrace']]
reg2000 = sm.OLS(Y2000,X2000).fit()
print(reg2000.summary())

                            OLS Regression Results                            
Dep. Variable:                 lhwage   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     147.8
Date:                Mon, 04 Oct 2021   Prob (F-statistic):          3.39e-138
Time:                        21:34:35   Log-Likelihood:                -2043.0
No. Observations:                2580   AIC:                             4098.
Df Residuals:                    2574   BIC:                             4133.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2952      0.217     -1.363      0.1

# MLE vs OLS result for 2000: Both in MLE and OLS the coefficient on education is 0.111!